ex1 : cv score : 0.015197305813776704  
..........................................................................................................  
ex2:  
not scaling, remove vehicle, not use nonscored_target  : 0.015448837474049428  
not scaling, remove vehicle, not use nonscored_target, 7 seeds  : cv score : 0.015381427768256765  
not scaling, remove vehicle, not use nonscored_target, drop high corr cols  : 0.01545810170388885
not scaling, remove vehicle, not use nonscored_target, add 95% , 60% : 0.015533423204263394
not scaling, remove vehicle, not use nonscored_target, add 95% , 50% : 0.015430995160848253
not scaling, remove vehicle, not use nonscored_target, add 95% , 40% : 0.015354553654126092
all : 0.01515220913930692(7 seeds)
all : 0.015207047278156766 (10 seeds)
all : model tuning + 3 layer (7seeds) cv score : 0.014899280863407222,  public : 0.01877 (←　0.01907, target 0.1840)

混ぜるほど良いスコア。。。？  
high corr な特徴落としたら少し悪化
95%追加で少し改善、なんか全部やった方がいい説


In [2]:
import warnings, random, os, sys
sys.path.append("../")
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, RobustScaler

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import tensorflow as tf

#from Models.DNN import DNN_model




pd.set_option("display.max_columns", 1200)
pd.set_option("display.max_rows", 1200)
%matplotlib inline

def metric(y_true, y_pred):
    res = []
    for i in range(0, y_true.shape[1]):
        y = y_true[:,i]
        pred = y_pred[:,i]
        res.append(log_loss(y, pred))
    return np.mean(res)

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    
def make_scaler(flag):
    if flag == "quantile":
        return QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    elif flag == "standard":
        return StandardScaler()
    elif flag == "minmax":
        return MinMaxScaler()
    elif flag == "robust":
        return RobustScaler()
    
seeds = [0, 1, 2, 3, 4]
SCALE = "quantile"


In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential, regularizers
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Concatenate, Embedding, Activation, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from tensorflow_addons.layers import WeightNormalization
from tensorflow_addons.optimizers import AdamW, Lookahead


def DNN_3lmodel(input_size, output_size):
    inp = Input(shape = (input_size, ))
    
    x = BatchNormalization()(inp)
    x = Dropout(0.4914099166744246)(x)
    x = WeightNormalization(Dense(1159, activation = 'relu'))(x)
    
    x = BatchNormalization()(x)
    x = Dropout(0.18817607797795838)(x)
    x = WeightNormalization(Dense(960, activation = 'relu'))(x)
    
    x = BatchNormalization()(x)
    x = Dropout(0.12542057776853896)(x)
    x = WeightNormalization(Dense(1811, activation = 'relu'))(x)
    
    x = BatchNormalization()(x)
    x = Dropout(0.20175242230280122)(x)
    out = WeightNormalization(Dense(output_size, activation = 'sigmoid'))(x)
    
    model = Model(inputs = inp, outputs = out)
    opt = Adam(learning_rate = 0.001)
    opt = Lookahead(opt, sync_period = 10)
    model.compile(
        optimizer=opt, 
        loss=BinaryCrossentropy(label_smoothing = 0.0015),
    )
    return model

def DNN_model(input_size, output_size):
    inputs = Input((input_size, ))

    outputs = BatchNormalization()(inputs)
    outputs = Dropout(0.20)(outputs)
    outputs = WeightNormalization(Dense(1024, activation="relu"))(outputs)

    outputs = BatchNormalization()(outputs)
    outputs = Dropout(0.20)(outputs)
    outputs = WeightNormalization(Dense(1024, activation="relu"))(outputs)

    outputs = BatchNormalization()(outputs)
    outputs = WeightNormalization(Dense(output_size, activation="sigmoid"))(outputs)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    model.compile(
        optimizer=AdamW(lr=0.001, weight_decay=1e-5, clipvalue=756), 
        loss=BinaryCrossentropy(label_smoothing = 0.0015)
    )

    return model


In [4]:
# g772, c100, 206クラス、402クラスの分類

train_df = pd.read_csv("../../Data/Raw/train_features.csv")
test_df = pd.read_csv("../../Data/Raw/test_features.csv")

y = pd.read_csv("../../Data/Raw/train_targets_scored.csv")
y_non = pd.read_csv("../../Data/Raw/train_targets_nonscored.csv")
y_all = pd.concat([y, y_non.drop("sig_id", axis=1)], axis=1)

GENES = [col for col in train_df.columns if col.startswith("g-")]
CELLS = [col for col in train_df.columns if col.startswith("c-")]
CELLS_50 = CELLS[:50]
BIOS = GENES + CELLS

SCORED_MOAS = [col for col in y.columns if col != "sig_id"]
NONSCORED_MOAS = [col for col in y_non.columns if col != "sig_id"]
ALL_MOAS = SCORED_MOAS + NONSCORED_MOAS

train_sigid = train_df["sig_id"]
test_sigid = test_df["sig_id"]

TR_SIZE = train_df.shape[0]
TE_SIZE = test_df.shape[0]

train_nonvehicle_index = train_df[train_df["cp_type"] != "ctl_vehicle"].index
test_nonvehicle_index = test_df[test_df["cp_type"] != "ctl_vehicle"].index

train_df["time_dose"] = train_df["cp_time"].astype(str) + " * " + train_df["cp_dose"]
test_df["time_dose"] = test_df["cp_time"].astype(str) + " * " + test_df["cp_dose"]

In [5]:
print(TE_SIZE)

3982


In [ ]:
"""g_li = [['g-616', 'g-48'], ['g-432',  'g-360',  'g-135',  'g-257',  'g-503',  'g-664',  'g-486',  'g-498',  'g-491',  'g-433',  'g-761'], ['g-443', 'g-566', 'g-615'], ['g-597', 'g-256'], ['g-90'], ['g-406'], ['g-525', 'g-272'], ['g-235'], ['g-392', 'g-708', 'g-689'], ['g-122', 'g-175', 'g-202'], ['g-547'], ['g-392', 'g-423', 'g-122'], ['g-302'], ['g-27', 'g-10', 'g-266'], ['g-635', 'g-745', 'g-108', 'g-626', 'g-0', 'g-8'], ['g-392', 'g-229', 'g-117', 'g-718'], ['g-566'], ['g-424', 'g-207', 'g-39', 'g-768', 'g-317'], ['g-392', 'g-206', 'g-600'], ['g-466', 'g-449'], ['g-2'], ['g-392',  'g-90',  'g-113',  'g-744',  'g-135',  'g-353',  'g-91',  'g-65',  'g-724',  'g-414'], ['g-443', 'g-322', 'g-211'], ['g-753', 'g-7', 'g-401', 'g-593', 'g-291'], ['g-178', 'g-435'], ['g-322', 'g-206'], ['g-27', 'g-600'], ['g-431',  'g-522',  'g-75',  'g-207',  'g-119',  'g-279',  'g-22',  'g-47',  'g-167',  'g-102',  'g-769',  'g-221'], ['g-104', 'g-587'], ['g-322', 'g-142'], ['g-569', 'g-690'], ['g-450'], ['g-241'], ['g-335'], ['g-206'], ['g-91', 'g-90'], ['g-606', 'g-484'], ['g-731', 'g-105'], ['g-596'], ['g-537', 'g-479'], ['g-446'], ['g-739'], ['g-466', 'g-38'], ['g-482', 'g-197', 'g-122'], ['g-150', 'g-29', 'g-570', 'g-58', 'g-764'], ['g-48'], ['g-466', 'g-428'], ['g-699',  'g-207',  'g-594',  'g-614',  'g-124',  'g-150',  'g-409',  'g-179',  'g-146',  'g-616',  'g-492'], ['g-70', 'g-628'], ['g-153'], ['g-75', 'g-696', 'g-431', 'g-102', 'g-207', 'g-697'], ['g-546', 'g-524', 'g-470'], ['g-628', 'g-701', 'g-376'], ['g-381', 'g-718', 'g-642'], ['g-417'], ['g-355'], ['g-94', 'g-636', 'g-335'], ['g-402', 'g-133'], ['g-368', 'g-481'], ['g-357'], ['g-122', 'g-210', 'g-206'], ['g-500'], ['g-355', 'g-201', 'g-50', 'g-63'], ['g-84', 'g-392', 'g-135'], ['g-615', 'g-417'], ['g-594'], ['g-362'], ['g-344', 'g-82', 'g-351'], ['g-134'], ['g-421'], ['g-29', 'g-47', 'g-670', 'g-117', 'g-525', 'g-36'], ['g-153', 'g-740', 'g-138'], ['g-601', 'g-664'], ['g-138', 'g-170', 'g-414', 'g-17', 'g-75', 'g-84', 'g-117'], ['g-405', 'g-583', 'g-367'], ['g-699',  'g-492',  'g-174',  'g-165',  'g-150',  'g-664',  'g-48',  'g-323',  'g-290',  'g-410',  'g-720',  'g-724',  'g-81'], ['g-75', 'g-178'], ['g-470', 'g-311', 'g-200'], ['g-392', 'g-175', 'g-593', 'g-14'], ['g-60', 'g-497'], ['g-169', 'g-29'], ['g-623', 'g-394', 'g-465'], ['g-533'], ['g-373'], ['g-392', 'g-175'], ['g-329', 'g-574', 'g-536', 'g-206'], ['g-431', 'g-128', 'g-75', 'g-187'], ['g-258', 'g-630', 'g-226'], ['g-36', 'g-675', 'g-207', 'g-761', 'g-554', 'g-59'], ['g-345', 'g-489']]
for li in g_li:
    if len(li) <= 1:
        continue
    for i in range(len(li)-1):
        for j in range(i+1, len(li)):
            col1 = li[i]
            col2 = li[j]
            train_df["p-"+col1+"-"+col2] = train_df[col1] * train_df[col2]
            test_df["p-"+col1+"-"+col2] = test_df[col1] * test_df[col2]
PRODS = [col for col in train_df.columns if col.startswith("p-")]
BIOS += PRODS"""

In [ ]:
"""
drop_cols = [
    'g-37', 'g-38', 'g-50', 'g-63', 'g-121', 'g-123', 'g-195', 'g-228', 'g-248', 'g-261', 'g-328', 'g-349', 
    'g-369', 'g-460', 'c-1', 'c-2', 'c-4', 'c-6', 'c-8', 'c-10', 'c-11', 'c-13', 'c-17', 'c-18', 'c-26', 'c-31', 'c-33', 
    'c-38', 'c-40', 'c-42', 'c-51', 'c-52', 'c-55', 'c-60', 'c-66', 'c-73', 'c-75'
]
train_df.drop(columns=drop_cols, inplace=True)
test_df.drop(columns=drop_cols, inplace=True)

BIOS = list(set(BIOS) - set(drop_cols))"""

In [ ]:
# 片側95%に入ったときに50%以上の確率で判別できるMoAの組み合わせが存在した列
n_cols = ['g-270', 'g-170', 'g-718', 'g-163', 'g-377', 'g-712', 'g-529', 'g-178', 'g-251', 'g-74', 'g-424', 'g-330', 'g-97', 'g-417', 'g-144', 'g-689', 'g-215', 'g-317', 'g-201', 'g-731', 'g-300', 'g-500', 'g-368', 'g-578', 'g-468', 'g-338', 'g-296', 'g-157', 'g-116', 'g-478', 'g-629', 'g-100', 'g-65', 'g-243', 'g-53', 'g-443', 'g-226', 'g-46', 'g-158', 'g-389', 'g-577', 'g-147', 'g-90', 'g-131', 'g-745', 'g-750', 'g-619', 'g-646', 'g-12', 'g-130', 'g-392', 'g-106', 'g-419', 'g-768', 'g-386', 'g-596', 'g-31', 'g-546', 'g-600', 'g-541', 'g-512', 'g-723', 'g-764', 'g-208', 'g-467', 'g-264', 'g-148', 'g-86', 'g-744', 'g-266', 'g-91', 'g-146', 'g-102']
p_cols = ['c-16', 'c-57', 'g-726', 'c-53', 'g-423', 'c-97', 'g-624', 'g-178', 'g-257', 'c-29', 'c-50', 'g-128', 'c-63', 'g-674', 'c-99', 'c-43', 'c-46', 'c-72', 'c-20', 'c-39', 'g-439', 'g-390', 'g-540', 'c-64', 'c-47', 'c-89', 'g-497', 'c-95', 'g-55', 'g-293', 'g-643', 'g-759', 'g-590', 'g-411', 'g-508', 'c-71', 'g-683', 'c-81', 'g-534', 'g-503', 'g-298', 'g-760', 'g-231', 'g-300', 'c-82', 'c-22', 'g-276', 'c-92', 'g-306', 'c-86', 'c-59', 'c-88', 'c-3', 'g-250', 'g-67', 'g-428', 'g-691', 'c-19', 'g-489', 'g-667', 'c-34', 'c-54', 'c-27', 'c-44', 'g-385', 'c-62', 'g-769', 'g-704', 'g-353', 'c-68', 'c-84', 'c-7', 'g-332', 'c-28', 'g-735', 'c-98', 'c-12', 'c-78', 'c-24', 'c-79', 'g-558', 'g-491', 'c-65', 'g-644', 'g-370', 'c-48', 'c-41', 'c-56', 'c-30', 'c-85', 'c-21', 'g-40', 'c-94', 'c-49', 'c-35', 'c-96', 'c-5', 'c-61', 'g-743', 'g-62', 'g-8', 'c-93', 'c-37', 'c-70', 'g-705', 'g-438', 'c-0', 'c-36', 'c-90', 'g-66', 'g-185', 'g-421', 'g-291', 'g-321', 'c-80', 'g-75', 'g-202', 'g-685', 'c-67', 'c-83', 'g-161', 'c-15', 'g-568', 'c-32', 'g-375', 'g-513', 'g-72', 'g-58', 'c-87', 'c-25', 'g-477', 'g-406', 'c-14', 'c-77', 'g-113', 'g-588', 'g-672', 'g-112', 'g-204', 'c-91', 'c-45']
for col in n_cols:
    train_df["n_"+col] = train_df[col].map(lambda x : 1 if x <= -9.5 else 0)
    test_df["n_"+col] = test_df[col].map(lambda x : 1 if x <= -9.5 else 0)
    
for col in p_cols:
    train_df["p_"+col] = train_df[col].map(lambda x : 1 if x >= 9.5 else 0)
    test_df["p_"+col] = test_df[col].map(lambda x : 1 if x >= 9.5 else 0)

In [ ]:
for val in train_df["time_dose"].unique():
    temp = pd.concat([train_df[train_df["time_dose"] == val], test_df[test_df["time_dose"] == val]])
    scaler = make_scaler(SCALE)
    scaler.fit(temp[BIOS].values.reshape(-1, len(BIOS)))
    tr_ind = train_df[train_df["time_dose"] == val].index
    te_ind = test_df[test_df["time_dose"] == val].index
    tr_trans = scaler.transform(train_df[BIOS].iloc[tr_ind].values.reshape(-1, len(BIOS))).T
    te_trans = scaler.transform(test_df[BIOS].iloc[te_ind].values.reshape(-1, len(BIOS))).T
    print("transformed")
    for i, col in enumerate(BIOS):
        train_df[col].iloc[tr_ind] = tr_trans[i]
        test_df[col].iloc[te_ind] = te_trans[i]

drop_cols = ["cp_time", "cp_dose", "time_dose"]

train_df = pd.concat([pd.get_dummies(train_df["time_dose"], prefix="onehot"), train_df.drop(drop_cols, axis=1) ], axis=1)
test_df = pd.concat([pd.get_dummies(test_df["time_dose"], prefix="onehot"), test_df.drop(drop_cols, axis=1) ], axis=1)




# remove cp_type = ctl_vehicle
print(train_df.shape)
print(test_df.shape)
print()

mask = train_df["cp_type"] != "ctl_vehicle"
train_df = train_df[mask].drop("cp_type", axis=1).reset_index(drop=True)
test_df = test_df[test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)

In [ ]:
X = train_df.drop("sig_id", axis=1)
test_X = test_df.drop("sig_id", axis=1)
y_nonv = y[mask].drop("sig_id", axis=1).values
y = y.drop("sig_id", axis=1).values

print(X.shape)
print(test_X.shape)
print(y_nonv.shape)

In [ ]:
train_preds = np.zeros((X.shape[0], y_nonv.shape[1]))
preds = np.zeros((test_X.shape[0], y_nonv.shape[1]))

for seed in seeds:
    seed_everything(seed)
    K = 5
    kf = MultilabelStratifiedKFold(n_splits=K, random_state=seed, shuffle=True)
    train_pred = np.zeros(train_preds.shape)
    
    for itr, (train_index, valid_index) in enumerate(kf.split(X, y_nonv)):
        print("======================== fold {} ========================".format(itr+1))
        train_X = X.iloc[train_index].values
        train_y = y_nonv[train_index]
        valid_X = X.iloc[valid_index].values
        valid_y = y_nonv[valid_index]
        
        model = DNN_3lmodel(input_size=train_X.shape[1], output_size=train_y.shape[1])
        #model = DNN_model(input_size=train_X.shape[1], output_size=train_y.shape[1])
        cb = EarlyStopping(monitor='val_loss', patience=1)
        
        early_stopping = EarlyStopping(monitor='val_loss',
                                                          mode = 'min',
                                                          patience = 10,
                                                          restore_best_weights = False,)
        
        reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                                         mode = 'min',
                                                         factor = 0.3,
                                                         patience = 3,)
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint(f'_{itr}_{seed}.h5',
                                                        monitor='val_loss',
                                                        save_best_only = True,
                                                        save_weights_only = True)
        
        
        model.fit(
            train_X, 
            train_y,
            batch_size=128,
            epochs=80,
            verbose=1,
            #callbacks=[cb],
            callbacks = [early_stopping, reduce_lr,  checkpoint],
            validation_data=(valid_X, valid_y),
        )
        model.load_weights(f'_{itr}_{seed}.h5')
        
        train_pred[valid_index] += model.predict(valid_X, batch_size=128)
        preds += model.predict(test_X.values, batch_size=128) / (K*len(seeds))


    print("seed {} , cv score : {}".format(seed, metric(y_nonv, train_pred)))
    train_preds += train_pred/len(seeds)

print("cv score : {}".format(metric(y_nonv, train_preds)))

In [ ]:
train_preds2 = np.zeros((TR_SIZE, y.shape[1]))
preds2 = np.zeros((TE_SIZE, y.shape[1]))

train_preds2[train_nonvehicle_index] = train_preds
preds2[test_nonvehicle_index] = preds

print("cv score : {}".format(metric(y, train_preds2)))

In [ ]:
preds = [:,:tag_size]
sub_df = pd.read_csv("../Data/Raw/sample_submission.csv")
#sub_df = pd.read_csv("../input/lish-moa/sample_submission.csv")
cols = [col for col in sub_df.columns if col != "sig_id"]
sub_df[cols] = preds2
#sub_df.to_csv("submission.csv", index=False)

In [ ]:
t = train_preds.copy()
print(metric(y[:,:tag_size], t[:,:tag_size]))
t_ = train_df[train_df["cp_type"] == 0]
t[t_.index] = np.zeros((t_.shape[0], t.shape[1]))
t = np.where(t > 1, 1, t)
print(metric(y[:,:tag_size], t[:,:tag_size]))

In [ ]:
li = []
not_li = []
for i in range(y.shape[0]):
    for j in range(y.shape[1]):
        if y[i][j] == 1:
            #print("====={}, {}====".format(i,j))
            rank = np.where(train_preds[i].argsort()[::-1] == j)[0][0]+1
            #print("rank {}".format(rank))
            if rank <= 20:
                li.append(j)
            else:
                not_li.append(j)
            #print(train_preds[i][j])

In [ ]:
li = list(set(li))
not_li = list(set(not_li))
for i in li:
    if i not in not_li:
        print(i)

In [ ]:
cols = pd.read_csv("../Data/Raw/train_targets_nonscored.csv").columns
pd.read_csv("../Data/Raw/train_targets_nonscored.csv")[cols[256-206]]